### initialize a config

In [ ]:
from amadeusgpt import AMADEUS
from amadeusgpt.config import Config
import amadeusgpt
from amadeusgpt.utils import parse_result
from pathlib import Path
import matplotlib.pyplot as plt
import cv2
from amadeusgpt.behavior_analysis.identifier import Identifier
from amadeusgpt.programs.task_program_registry import TaskProgramLibrary

In [ ]:
amadeus_root = Path(amadeusgpt.__file__).parent.parent
config = Config(amadeus_root / "amadeusgpt/configs/MABe_template.yaml")
config['data_info']['data_folder'] = amadeus_root / config['data_info']['data_folder']

amadeus = AMADEUS(config)
video_file_paths = amadeus.get_video_file_paths()
print (video_file_paths)

### Register a new task program into the library

In [ ]:
@TaskProgramLibrary.register_task_program(creator="human")
def get_approach_events(identifier : Identifier):
    """
    This task program describes the approach events between any pair of two animals.
    """
    # behavior_analysis was defined in the namespace. Just take this as syntax
    analysis = create_analysis(identifier)    
    orientation_events = analysis.event_manager.get_animals_animals_events(['relative_head_angle <= 30'])
                                                                

    close_events = analysis.event_manager.get_animals_animals_events(['distance < 300', 'relative_speed > 0.0'])

        
    speed_events = analysis.event_manager.get_animals_state_events("speed > 0.1")


    close_speed_events = analysis.event_manager.get_composite_events(close_events,
                                                                    speed_events,
                                                                    "logical_and")
    
    approach_events = analysis.event_manager.get_composite_events(close_speed_events,
                                                                    orientation_events,
                                                                    "logical_and")
    
  
    return approach_events

In [ ]:
@TaskProgramLibrary.register_task_program(creator="human")
def get_corner_events(identifier : Identifier):
    # behavior_analysis is already defined in the namespace
    analysis = create_analysis(identifier)
    roi0_events = analysis.event_manager.get_animals_object_events('ROI0', 'overlap==True')
    roi1_events = analysis.event_manager.get_animals_object_events('ROI1', 'overlap==True')
    roi2_events = analysis.event_manager.get_animals_object_events('ROI2', 'overlap==True')
    roi3_events = analysis.event_manager.get_animals_object_events('ROI3', 'overlap==True')
    roi0_or_1_events = analysis.event_manager.get_composite_events(roi0_events, roi1_events, 'logical_or')
    roi2_or_3_events = analysis.event_manager.get_composite_events(roi2_events, roi3_events, 'logical_or')
    corner_events = analysis.event_manager.get_composite_events(roi0_or_1_events, roi2_or_3_events, 'logical_or')
    return corner_events

In [ ]:
amadeus = AMADEUS(config)

In [ ]:
behavior_analysis = amadeus.get_behavior_analysis(video_file_path='/Users/shaokaiye/AmadeusGPT-dev/examples/MABe/EGS8X2MN4SSUGFWAV976.mp4')
scene_image = behavior_analysis.visual_manager.get_scene_image()
plt.imshow(scene_image)

In [ ]:
qa_message = amadeus.run_task_program('get_approach_events')
parse_result(amadeus, qa_message)

In [ ]:
amadeus.save_results()

In [ ]:
amadeus.load_results()